# Univariate Gaussian MLE

## Problem Statement

I have $n=100$ pieces of indepdendent and identicially distrubuted (i.i.d) data related to some measurement that is drawn from a **normal distribution**.

Your task is to write code that computes $\mu_{MLE}$ and $\sigma_{MLE}$, which represent the mean and standard deviation computed via MLE, respectively, of the underlying normal distribution for the dataset "MLE_dataset.npy"

## Variable Definitions

1. x = collection of data points ($x_1,x_2,...,x_n$). Loaded in as a numpy array of shape (100,)
2. $\mu$ = mean of the normal distribution
3. $\sigma$ = standard deviation of the normal distribution

## Useful Numpy functions

1. x.shape : returns a tuple that contains the dimension(s) of the numpy array variable name "x"

## Starting information

You are given "MLE_dataset.npy" which contains the aforementioned data in the form of a numpy array.

You are also given a "load" function which, given the path to "MLE_dataset.npy", will load the values into a numpy array variable and return said variable.

For a single piece of data, $x_i$, if the data is drawn from a normal distribution, the likelihood of that data given some values $\mu$ and $\sigma^2$ can be represented as:

$$p(x_i | \mu, \sigma^2) = \mathcal{N}(x_i; \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} * \text{exp} \ [-\frac{1}{2}(\frac{x_i - \mu}{\sigma})^2]$$

In our case, we aren't dealing with a single observation, but $n=100$ observations. Because our observations are **independent**, we can represent our likelihood function for all observations as the product of the individual observations:

$$p(x | \mu, \sigma^2) = \Pi_{i=1}^N \mathcal{N}(x_i; \mu, \sigma^2) = (\frac{1}{2\pi\sigma^2})^{n/2} * \text{exp} \ [-\frac{1}{2}\sum_{i=1}^N (\frac{x_i - \mu}{\sigma})^2]$$


## Task

Using the starting information, your task is to:
1. Derrive the formula for $\mu_{MLE}$
2. Derrive the formula for $\sigma_{MLE}$
3. Write code based on your answers to (1) and (2) that correctly finds $\mu_{MLE}$ and $\sigma_{MLE}$ for the data set "MLE_dataset.npy"

## MLE estimates for $\mu$ and $\sigma^2$ (not given to students)

$$\mu_{MLE} = \frac{1}{n} \sum_{i=1}^n x_i$$

$$ \sigma^2_{MLE} = \frac{1}{n} \sum_{i=1}^n(x_i-\mu_{MLE})^2$$

In [ ]:
#Not given to students

#generation of data points with mu_true = 100, sigma_true = 10
'''
n = 100 #number of data points
mu_true = 100
sigma_true = 10
dataset = []
for i in range(n):
    dataset.append(np.random.normal(mu_true,scale=sigma_true))
dataset = np.asarray(dataset)

#np.save("./MLE_dataset.npy",dataset)

'''

In [3]:
import scipy.stats
import numpy as np


def load(path):
    '''
    loads "MLE_dataset.npy" given its path into variable 'dataset'
    '''
    dataset = np.load(path)
    return dataset
    
def MLE(dataset):
    mu = 0
    sigma = 0
    
    ''' YOUR CODE HERE '''
    for i in range(dataset.shape[0]):
        mu += dataset[i]
    mu /= dataset.shape[0]
    
    
    for i in range(dataset.shape[0]):
        sigma += (dataset[i] - mu)**2
    sigma /= dataset.shape[0]
    sigma = np.sqrt(sigma)
    
    ''' END YOUR CODE'''
    
    return mu, sigma

'''YOUR TEST CODE HERE'''
dataset = np.load("./MLE_dataset.npy")
mu_guess, sigma_guess = MLE(dataset)
print("mu_estimate: ", mu_guess, " || sigma estimate: ", sigma_guess )

mu_estimate:  102.65639598061982  || sigma estimate:  9.743253619192297


# Univariate Gaussian EM

## Problem statement

Let us add a layer of difficulty to the problem of estimating the underlying distribution(s) of our data. Previously. we knew that all of our data came from a single underlying normal distribution.

For this problem, I have gone out and collected a brand new data set consisting of $n=200$ data points. These data points are drawn from **one of two unknown gaussian distributions**.

To keep the notation consistent, I will use the subscript ID $k = \{0,1\}$ to represent which gaussian a parameter/variable is referring to.

For instance, $\mu_k$ represents the mean of the gaussian with ID $k$. $\mu_0$ represents the mean of the first gaussian, and $\mu_1$ represents the mean of the second gaussian

## Variable definitions

1. $k$: id of the gaussian distributions. {0,1} 
2. $n$: number of data points. 200 in this case
3. $\mu_k$: the mean of normal distribution $k$
4. $\sigma_k$: the std of normal distribution $k$
5. $\pi_k$: the prior probability of normal distribution $k$
7. $t$: current step number
8. $z_i = k$: represents the idea that i-th data point was drawn from gaussian $k$
9. $x_i$: i-th data point

## Task Description:

To make things clear, here is a list of things we do and don't know.

**Do know:**
1. Our data points, $x = x_1, x_2, ..., x_n$ for $n = 1$ to $n=200$
2. $\mu_0^{t=0}, \sigma_0^{t=0}, \pi_0^{t=0}$ which represent an initial guess for the  mean, std, and prior  of the first gaussian with ID $k=0$
3. $\mu_1^{t=0}, \sigma_1^{t=0}, \pi_1^{t=0}$ which represent an initial guess for the  mean, std, and prior  of the second gaussian with ID $k=1$
4. $\pi_0^{t=0}$ and $\pi_1^{t=0}$ which represent an initial guess for the prior probabilities of gaussian 0 and gaussian 1, respectively
5. The number of iterations your EM algorithm should run for.
6. Let us define the variable $\theta_k^t$ to be the set ($\hat{\mu_k^t}, \hat{\sigma_k^t}, \hat{\pi_k^t})$, which is our current estimate at time step t for the three unknown parameters of gaussian k. 

**Don't know:**
1. The true means, $\mu_0$ and $\mu_1$ for both gaussian distributions
2. The true standard deviations, $\sigma_0$ and $\sigma_1$ for both gaussian distributions 
3. Which gaussian distribution, $k=0$ or $k=1$, a point, $x_i$, came from.
4. The true fraction of our points that came from gaussian $k=0$ and the true fraction of our points that came from gaussian $k=1$


Your end goal is to implement an algorithm that implements EM and returns the following:
1. $\hat{\mu_0}, \hat{\sigma_0}, \hat{\pi_0}$ which represent the final EM estimate the mean, standard deviation (std), and prior of the first gaussian with ID $k=0$
2. $\hat{\mu_0}, \hat{\sigma_0}, \hat{\pi_0}$ which represent the final EM estimate of the mean, standard deviation (std), and prior of the first gaussian with ID $k=1$

### Recommended task order:

1. Write code that implements E step 
2. Write code that implements M step 


## E - Step - should I give them this info?
1. $P(z_i = k | x_i, \theta_k^t)$ reflects the responsibility the k-th class has for the i-th data point
$$P(z_i = k | x_i, \theta_k^t) = \frac{P(x_i | z_i = k, \theta_k^t)*\pi_k}{P(x_i | \theta_k^t)}$$ \
$$= \frac{P(x_i | z_i = k, \theta_k^t)*\pi_k}{\sum_{k=0}^{1}P(x_i | z_i = k, \theta_k^t)*\pi_k}$$



## M - Step - should I give them this info?


$$\mu_k^{t+1} = \frac{\sum_{i=1}^n x_i * P(z_i = k | x_i, \theta_k^t)}{\sum_i^n P(z_i = k | x_i, \theta_k^t)}$$


$$\sigma_k^{t+1} = (\frac{\sum_{i=1}^n(\mu_k^{t+1} - x_i)^2 P(z_i = k | x_i, \theta_k^t)}{\sum_{i=1}^{n}P(z_i = k | x_i, \theta_k^t)})^{1/2}$$


$$\pi_k^{t+1} = \frac{\sum_{i=1}^n P(z_i = k | x_i, \theta_k^t)}{n}$$

In [ ]:
#not given to students
#data generation for EM
'''
class_probs_true = [0.4, 0.6]
mus_true = [100, 200]
sigmas_true = [24, 21]
random_seed = None # for reproducability
n_samples = 200
n_iterations = 100
n_classes = 2

dataset = []
for c in range(len(class_probs_true)):
    for i in range(int(class_probs_true[c]*n_samples)):
        dataset.append(np.random.normal(mus_true[c],scale=sigmas_true[c]))

dataset = np.asarray(dataset)

np.save("EM_dataset.npy", dataset)
'''

In [1]:
import scipy.stats
import numpy as np

def load(path):
    dataset = np.load(path)
    return dataset

def em(dataset, k, n_iterations):
    '''
    Input:
        dataset - np array containing the data
        k - int representing the number of underlying gaussian distributions
        n_iterations - int representing number of iterations EM should run for
        
    output:
        
    '''
    #n_classes = k
    #n_samples = n
    n_samples = dataset.shape[0]

    # Initial guesses for the parameters
    mus = np.asarray([90, 210])
    sigmas = np.asarray([28,19])
    pi = np.asarray([0.3,.7])
    
    print("initial mu: ", mus)
    print("initial sigmas:  ", sigmas)
    print("initial pi: ", pi)
            
    for em_iter in (range(n_iterations)):
            print("")
            print('-------iter: ', em_iter, '------------------------')
            
            
            #E Step
            #ws = responsibilities of each gaussian for each data point. shape = (k,n_samples)
            ws = np.zeros((k, dataset.shape[0]))
            for i in range(n_samples):
                for c in range(mus.shape[0]):
                    class_likelihood = scipy.stats.norm(mus[c], sigmas[c]).pdf(dataset[i])*pi[c]
                    ws[c, i] = class_likelihood
            #print(k0)
            #print(ws[1,:])
            #assert 1== 0
            #normalize ws for each class
            original_ws = np.copy(ws)
            ws[0,:] = ws[0,:]/(original_ws[0,:] + original_ws[1,:])
            ws[1,:] = ws[1,:]/(original_ws[0,:] + original_ws[1,:])
            
            #m step
            '''
            using the normalized ws from above, we can compute our next guess for pis, mus, and sigmas
            
            '''
            pis = np.zeros(k) 
            for j in range(mus.shape[0]): #for each mean
                for i in range(n_samples): #iterate over all samples and sum up their ws
                    pis[j] += ws[j,i]
            pis /= n_samples
            print("pis: ", pis)
            
            mus = np.zeros((k))
            for j in range(k):
                for i in range(n_samples):
                    mus[j] += ws[j, i] * dataset[i]
                mus[j] /= ws[j,:].sum()
            print("mus: ", mus)
            
            #print('sums: ', ws[0].sum(), ws[1].sum())
            #assert 1 == 0
            sigmas = np.zeros(k)
            for j in range(k):
                for i in range(n_samples):
                    ys = dataset[i]-mus[j]
                    sigmas[j] += ws[j, i] * ys**2
                sigmas[j] /= ws[j].sum()
            for s in range(sigmas.shape[0]):
                sigmas[s] = np.sqrt(sigmas[s])
            print("sigmas: ", sigmas)

    return pi, mus, sigmas

def main():
    n_samples = 200
    n_iterations = 15
    k = 2
    
    #true values will not be given to students
    class_probs_true = [0.4, 0.6]
    mus_true = [100, 200]
    sigmas_true = [24, 21]
    random_seed = None # for reproducability

    dataset = np.load("EM_dataset.npy")
#     dataset = []
#     for c in range(len(class_probs_true)):
#         for i in range(int(class_probs_true[c]*n_samples)):
#             dataset.append(np.random.normal(mus_true[c],scale=sigmas_true[c]))

#     #dataset = np.asarray(dataset)

    print("True pis: ", class_probs_true)
    print("True mus: ", mus_true)
    print("True sigmas: ", sigmas_true)

    class_probs, mus, sigmas = em(dataset, k, n_iterations)
    
    print("True pis: ", class_probs_true)
    print("True mus: ", mus_true)
    print("True sigmas: ", sigmas_true)

    
main()

True pis:  [0.4, 0.6]
True mus:  [100, 200]
True sigmas:  [24, 21]
initial mu:  [ 90 210]
initial sigmas:   [28 19]
initial pi:  [0.3 0.7]

-------iter:  0 ------------------------
pis:  [0.42194944 0.57805056]
mus:  [101.37853784 202.42662628]
sigmas:  [23.98148408 19.30987141]

-------iter:  1 ------------------------
pis:  [0.41486436 0.58513564]
mus:  [100.44327673 201.86619613]
sigmas:  [23.05069349 19.87583409]

-------iter:  2 ------------------------
pis:  [0.41088789 0.58911211]
mus:  [ 99.9228902  201.54455128]
sigmas:  [22.52267853 20.20569715]

-------iter:  3 ------------------------
pis:  [0.40869937 0.59130063]
mus:  [ 99.64273507 201.3620698 ]
sigmas:  [22.24247102 20.39816386]

-------iter:  4 ------------------------
pis:  [0.40751351 0.59248649]
mus:  [ 99.49329212 201.26126611]
sigmas:  [22.09484496 20.50651385]

-------iter:  5 ------------------------
pis:  [0.40687721 0.59312279]
mus:  [ 99.41386731 201.20657422]
sigmas:  [22.01700576 20.56596472]

-------iter:  